In [37]:
%load_ext autoreload
%autoreload 2

import os
import pymupdf
import numpy as np
import math

from text import extract_words, create_text_lines, create_text_blocks
from utils import classify_wordpos, classify_text_density


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
base_dir = os.getcwd()
input_folder = "maps"
filename ="8627_3.pdf"

pdf_path = os.path.join(base_dir, "data/input", input_folder)
out_dir = os.path.join(base_dir, "data/test", os.path.splitext(filename)[0])

# pdf_path = "/home/lillemor/PycharmProjects/swissgeol-boreholes-dataextraction/data/zurich"

In [55]:

def calculate_distance(word1, word2):
    """Calculate Euclidean distance between two TextWord objects based on x0 and y0"""
    x_dist = word1.rect.x0 - word2.rect.x0
    y_dist = word1.rect.y0 - word2.rect.y0
    return math.sqrt(x_dist**2 + y_dist**2)

def closest_word_distances(words):
    """Calculate distances between each word and its closest neighbor"""
    if not words or len(words) < 2:
        return []

    distances = []
    for i, word in enumerate(words):
        other_words = words[:i] + words[i+1:]  # Exclude current word
        closest_word = min(other_words, key=lambda w: calculate_distance(word, w))
        distances.append(calculate_distance(word, closest_word))

    return distances

In [59]:
from collections import defaultdict
import pymupdf

def y0_word_cluster(all_words, tolerance: int = 10):
   
    if not all_words:
        return []

    # Dictionary to hold clusters, keys are representative y0 values
    grouped_y0 = defaultdict(list)

    for word in all_words:
        y0 = word.rect.y0
        matched_y0 = None

        # Check if y0 is within tolerance of an existing cluster
        for key in grouped_y0:
            if abs(key - y0) <= tolerance:
                matched_y0 = key
                break

        # Add to an existing cluster or create a new one
        if matched_y0 is not None:
            grouped_y0[matched_y0].append(word)
        else:
            grouped_y0[y0].append(word)

    clusters = list(grouped_y0.values())

    return clusters

In [ ]:
count = 0
for filename in os.listdir(pdf_path):
    # if count >=1:
    #     break
    if filename.lower().endswith('.pdf'):
        file_path = os.path.join(pdf_path, filename)
        
        with pymupdf.Document(file_path) as doc: 
        
            for page_index,page in enumerate(doc):
                
                #page info
                page_number = page_index +1
                page_size = (page.rect.width, page.rect.height)

                #words attributes
                words = extract_words(page, page_number)
                if not words:
                    print("no words on page")
                    continue
                word_attributes = classify_text_density(words,page_size)
                words_position = classify_wordpos(words)
                distances = closest_word_distances(words)
                median_distance = np.median(distances) if distances else None

                #line attributes
                lines = create_text_lines(page, page_number)
                words_per_line =[len(line.words) for line in lines]

                #textblock attributes
                text_blocks = create_text_blocks(lines)
                block_area = sum([block.rect.get_area() for block in text_blocks])
                word_area =sum([word.rect.get_area() 
                    for block in text_blocks
                    for line in block.lines 
                    for word in line.words if len(line.words) > 1])
                
                if word_area/block_area > 1 and np.mean(words_per_line) >  3:
                    print("text")
 
                else:
                    ## cluster lines based on y0 ( could also do text_blocks...)
                    clusters = y0_word_cluster(lines)

                    ## filter out clusters with only one line
                    filtered_clusters = [cluster for cluster in clusters if len(cluster) >=2]
                    if filtered_clusters:
                        
                        ##check if we have long cluster with multiple entries -> boreprofile
                        print(sum(len(cluster) for cluster in filtered_clusters)/len(filtered_clusters))
                        print()
                    # print(len(lines))
                    else:
                        print("map")

                    #print("image")
                
                # if np.mean(words_per_line) <=  3:
                #     print(f"file: {filename}: map") 
                # else:
                #     print(f"file: {filename}: text") 

                #print(round(median_distance,2), np.mean(words_per_line)) ## maps higher euclidean distance between words than borehole or text
                count +=1


2.5

2.0

3.2244897959183674

2.5

map
2.3333333333333335

2.875

3.9

6.0

2.5714285714285716

3.0

3.0

2.25

3.3947368421052633

3.25

2.6666666666666665

4.9523809523809526

3.4057971014492754

2.3333333333333335

2.3333333333333335

2.5

3.0

2.2



In [42]:
import numpy as np
import matplotlib.pyplot as plt

from skimage import measure

# Construct some test data
x, y = np.ogrid[-np.pi : np.pi : 100j, -np.pi : np.pi : 100j]
r = np.sin(np.exp(np.sin(x) ** 3 + np.cos(y) ** 2))

# Find contours at a constant value of 0.8
contours = measure.find_contours(r, 0.8)